# PM2.5 Forecasting

In [2]:
import numpy as np

In [1]:
import optuna # Hyperparameter optimization
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

/Users/valeriagonzalez/Documents/aqi/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/valeriagonzalez/Documents/aqi/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Search hyperparameters

In [ ]:
class MLPOptimizer:
  """
  Class for hyperparameter optimizing for PM2.5 Forecaster.
  """
  def __init__(self, input_dim:int, n_trials:int):
    self.input_dim = input_dim
    self.n_trials = n_trials
    self.study = None
    self.X_train = None
    self.y_train = None
    self.X_val = None
    self.y_val = None

  def create_model(self, trial):
    model = Sequential()

    # Hyperparameter selection
    neurons = trial.suggest_categorical("neurons", [5, 50])
    dropout_rate = trial.suggest_float("dropout", 0.1, 0.2, log=True)
    
    model.add(LSTM(neurons, 
                   input_shape=(self.X_train.shape[1], self.X_train.shape[2]),
                   activation="relu"))


    # Dropout layer
    # model.add(Dropout(dropout_rate))

    # Output layer
    model.add(Dense(1))

    # Compile model
    # learning_rate = trial.suggest_cat("learning_rate", 0.001, 0., log=True)
    learning_rate = 0.001
    model.compile(optimizer=Adam(learning_rate), loss="mae")

    return model

  def objective(self, trial):
    model = self.create_model(trial)
    epochs = trial.suggest_categorical("epochs", [50, 100])

    # Train the model
    history = model.fit(
        self.X_train,
        self.y_train,
        epochs=epochs,
        batch_size=72,
        validation_data=(self.X_val, self.y_val),
        verbose=0
        )

    # Evaluate on validation data
    loss = model.evaluate(self.X_val, self.y_val, verbose=0)

    return loss

  def optimize(self, X_train:np.array, y_train:np.array, X_val, y_val:np.array):
    self.X_train = X_train
    self.y_train = y_train
    self.X_val = X_val
    self.y_val = y_val

    self.study = optuna.create_study(direction="minimize")
    self.study.optimize(self.objective, n_trials = self.n_trials)

    print("Best hyperparameters:", self.study.best_params)
    return self.study

In [ ]:
mlp_optimizer = MLPOptimizer(n_steps, 32)
study = mlp_optimizer.optimize(X_train, y_train, X_val, y_val)